In [1]:
import json
import os
import torch
from argparse import ArgumentParser

from pytorch_lightning import seed_everything

from transformers import EarlyStoppingCallback

from src.datasetComposer import DatasetBuilder, composed_train_path, composed_test_path, compactComposer, test_path, train_path, test_path
from src.inference_utils import InferenceGenerator
from src.datasetHandlers import SmartCollator
from src.model_utils import get_basic_model
from src.trainerArgs import CustomTrainer, getTrainingArguments
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:

iterative_gen = True
composed_already = True

# Define the parameters used to set up the models
modeltype = 'iterative' if iterative_gen else 'normal'  # either baseline or 'earlyfusion'

# either t5-small,t5-base, t5-large, facebook/bart-base, or facebook/bart-large
modelbase = 'facebook/bart-base'

# we will use the above variables to set up the folder to save our model
pre_trained_model_name = modelbase.split(
    '/')[1] if 'bart' in modelbase else modelbase

# where the trained model will be saved
output_path = 'TrainModels/' + modeltype + '/'+pre_trained_model_name+'/'

#tests = json.load(open(test_path,encoding='utf-8'))

## Dataset 
Only the data for the iterative generation have been processed already to reduce the loading and processing time. 


In [3]:
# Load the dataset
if iterative_gen:
    experiments_dataset = DatasetBuilder(train_data_path=composed_train_path,
                                     test_data_path=composed_test_path,
                                     modelbase= modelbase,
                                     iterative_mode= iterative_gen,
                                     composed_already=composed_already)
else:
    experiments_dataset = DatasetBuilder(train_data_path=train_path,
                                     test_data_path=test_path,
                                     modelbase= modelbase,
                                     iterative_mode= False,
                                     composed_already=False)


experiments_dataset.fit()

print('Dataset built')
print(f"Training size: {len(experiments_dataset.train_dataset)}")
print(f"Test size: {len(experiments_dataset.test_dataset)}")

Dataset built
Training size: 2464
Test size: 215


# Model
Below we create the narration model

In [8]:
from transformers import TrainingArguments

In [ ]:
TrainingArguments??

In [12]:
rand_seed = 453
seed_everything(rand_seed)
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

arguments = train_arguments = {'output_dir': output_path,
                               'warmup_ratio': 0.2,
                               #'disable_tqdm':False,
                               'per_device_train_batch_size': 8,
                               'num_train_epochs': 4,
                               'lr_scheduler_type': 'cosine',
                               'learning_rate': 5e-5,
                               'evaluation_strategy': 'steps',
                               'logging_steps': 500,
                               
                               'seed': rand_seed}

# Build actual trainingArgument object
training_arguments = getTrainingArguments(train_arguments)

Global seed set to 453
using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [13]:
# Define the model
getModel = get_basic_model(experiments_dataset)


trainer = CustomTrainer(model_init=getModel,
                        data_collator=SmartCollator(
                            pad_token_id=experiments_dataset.tokenizer_.pad_token_id),
                        args=training_arguments,
                        train_dataset=experiments_dataset.train_dataset,
                        eval_dataset=experiments_dataset.test_dataset,
                        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)])

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /home/essel/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,

In [14]:
trainer.train()

# Save the model with the lowest evaluation loss
trainer.save_model()
trainer.save_state()

# get the best checkpoint
best_check_point = trainer.state.best_model_checkpoint


params_dict = train_arguments

params_dict['best_check_point'] = best_check_point
params_dict['output_path'] = output_path
json.dump(params_dict, open(f'{output_path}/parameters.json', 'w'))

loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /home/essel/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.a243ed957122436adb0b8d8e9d20f896f45c174b6324d625ca0a20a84f72a910
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,

{'loss': 1.9543, 'learning_rate': 4.229302552505459e-05, 'epoch': 1.62}


Saving model checkpoint to TrainModels/iterative/bart-base/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 1.2657783031463623, 'eval_runtime': 0.878, 'eval_samples_per_second': 244.871, 'eval_steps_per_second': 30.751, 'epoch': 1.62}


***** Running Evaluation *****
  Num examples = 215
  Batch size = 8


{'loss': 1.2336, 'learning_rate': 6.537421729016493e-06, 'epoch': 3.25}


Saving model checkpoint to TrainModels/iterative/bart-base/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'eval_loss': 1.1422196626663208, 'eval_runtime': 0.8829, 'eval_samples_per_second': 243.521, 'eval_steps_per_second': 30.582, 'epoch': 3.25}


Deleting older checkpoint [TrainModels/iterative/bart-base/checkpoint-500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from TrainModels/iterative/bart-base/checkpoint-1000 (score: 1.1422196626663208).
Saving model checkpoint to TrainModels/iterative/bart-base/
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


{'train_runtime': 210.4055, 'train_samples_per_second': 46.843, 'train_steps_per_second': 5.855, 'train_loss': 1.5025448737206397, 'epoch': 4.0}


# Inference

In [11]:
inference_model = trainer.model.generator.to(device)
inference_model.eval();

In [12]:
tests = json.load(open(test_path, encoding='utf-8'))
# ,iterative_mode=False,force_section=False
if iterative_gen:
    test_examples = compactComposer(
        tests, iterative_mode=iterative_gen, force_section=True)
    test_examples2 = compactComposer(tests, iterative_mode=iterative_gen, force_section=False)
else:
    test_examples = compactComposer(tests, iterative_mode=iterative_gen, force_section=False)

# Instantiate the inference routine
iterativeGen = InferenceGenerator(inference_model,
                                  experiments_dataset,
                                  device,
                                  max_iter=8,
                                  sampling=False, verbose=False)


In [16]:
max_full_len = 300
if iterative_gen:
    print("Explanation Generation Iteratively")
    iterativeGen.sampling = False
    #output_sentences = iterativeGen.MultipleIterativeGeneratorJoint( test_examples, parsed_args.seed,)
    output_sentences = iterativeGen.MultipleIterativeGeneratorJoint(
        test_examples[:1], rand_seed, length_penalty=1.6,max_length=269)
    output_sentences2 = iterativeGen.MultipleIterativeGeneratorJoint(
        test_examples2[:1], rand_seed, length_penalty=1.6,max_length=269)
else:
    print("Full Explanation Generation")
    output_sentences = iterativeGen.MultipleFullGeneratorJoint(
        test_examples, rand_seed, max_length=max_full_len)

Global seed set to 453


Explanation Generation Iteratively


Global seed set to 453


Completed
Completed


In [ ]:
test_examples[:1][0]

In [19]:
output_sentences

["The most probable label for the given case is C1 with a predicted probability of 69.02%, which implies that there is a 30.98% chance that  C2 could be the correct label. F3, F2, and F8 are the features with the most influence on the classifier's decision. F5, F7, and F10 are the features that increase the likelihood of C1 being the correct label. F3, F2, and F8 are the positive features that increase the likelihood of C1 being the correct label. F5, F7, and F10 are the positive features that increase the odds of C1 being the correct label. On the other hand, F9 and F4 are shown to have very little impact on the classifier's decision here."]